<a href="https://colab.research.google.com/github/heimmer/NLP/blob/main/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quention 1  

## 1
**answer**  
- **vanishing/exploding gradient**:   
Loss function of RNN is defined as sum of the loss at every time step. After decomposing the loss at each time step according to the chain rule, $\frac{∂h_t}{∂h1}$ is the only term which is hard to compute, we need to further decompose it to $\frac{∂h_t}{∂h_{t-1}}\frac{∂h_{t-1}}{∂h_{t-2}}...\frac{∂h2}{∂h1}$. It's fine that each $\frac{∂h_t}{∂h_{t-1}}$ is a bit larger or smaller than 1, but in RNN, the time step T can be large. After a expotential calculation of power T, the gradient can be very large or small, which is the so-called vanishing/exploding gradient problem.  
- **long-term dependency**  
Human language system is complicated, sometimes information related to the t-th word appeared very early in the sentence. If the gradient is small, the model can't learn this dependency, which is so-called long-term dependency problem.

## 2  
**answer**  
Yes, LSTM can solve the problems of vanishing and exploding gradient.  
The recursive derivative in Vanilla RNN is the main reason of vanishing/exploding gradient.But LSTM introduces a set of gating units and a cell state to control the memory intead of adding up all history.  
- **forget gate** controls what is kept vs forgotten from previous cell state  
- **input gate** controls what part of new cell content are written to the cell
- **output gate** controls what parts of cell are output to hidden state  

Those gates can help to adjust and prevent the gradients from vanishing/exploding

## 3  
**answer** 
$4*[(d_i+d_o)*d_o+d_o]$  
- in each LSTM cell, there are 4 non-linear transformations (3 gates and 1 tanh)
- dim of input is $d_i$, dim of hidden state from previous LSTM cell is $d_o$
- at first, we need to combine them together, so dim of the combination is $d_i+d_o$  
- output of each non-linear transformation $σ$ is of dim $d_o$, and dim of bias is also $d_o$
- so number of parameters of each non-linear transformation is $(d_i+d_o)*d_o+d_o$
- 4 non-linear transformations are different, they don't share parameters, so number of parameters of the whole cell is $4*[(d_i+d_o)*d_o+d_o]$

## 4  
**answer** : 12585  
- vocab_size=1000, embedding_dim=10, so number of parameters in embedding matrix is 1000*10=10000
- embedding_dim=10, so input_size of LSTM cell is 10; hidden_size=20, according to last question, number of parameters in LSTM cell is 4*[(10+20)*20+20]=2480
- hidden_size=20, num_class=5, so number of parameters in decoder is 20*5+5=105
- total number of parameters is 10000+2480+105 = 12585

In [ ]:
import torch.nn as nn
class MyModel(nn.Module):
  def __init__(self, vocab_size=1000, embedding_dim=10,
        hidden_size=20, num_class=5):
    super(MyModel, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size,
    bidirectional=True, num_layers=1)
    self.decoder = nn.Linear(hidden_size, num_class, bias=True)
model = MyModel()

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7f7d03300ac0>

In [ ]:
for param in model.named_parameters():
    print(param[0])

embeddings.weight
lstm.weight_ih_l0
lstm.weight_hh_l0
lstm.bias_ih_l0
lstm.bias_hh_l0
lstm.weight_ih_l0_reverse
lstm.weight_hh_l0_reverse
lstm.bias_ih_l0_reverse
lstm.bias_hh_l0_reverse
decoder.weight
decoder.bias


In [ ]:
for param in model.parameters():
  print(param.shape)

torch.Size([1000, 10])
torch.Size([80, 10])
torch.Size([80, 20])
torch.Size([80])
torch.Size([80])
torch.Size([80, 10])
torch.Size([80, 20])
torch.Size([80])
torch.Size([80])
torch.Size([5, 20])
torch.Size([5])


# Quention 2

1  
**answer**  
advantages:
- solve the long-term dependency problem, after attention layer, each output will carry information from all inputs
- the calculation in attention layer goes parallelly as matrix multiplication, rather than sequentially, which is more computationally efficient  

disadvantages:  
- the sequential process of RNN naturally save position information, but attention layer won't save position information, it needs positional encoding to assist
- assume length of the sequence is $L$, then complexity of attention is $L^2$. If $L$ is too large, attention will become computationally inefficient

2  
**answer**  
For large values of $d_k$, the dot products grow large in magnitude, pushing the softmax into the regions where is has extremely small gradients. So it is necessary to introduce a scaler before normalization.  

Mathematically, 
- queries and keys are of dimension $d_k$, so $A=QK^T$ is a $d_k*d_k$ matrix
- its first element is $A_{1,1}=\sum\limits_{i=0}^{d_k}Q_{1,i}K_{i,1}$. Q and K are random variables, so each entry of them has 0 mean and 1 variance
- $E[Q_{1,i}K_{i,1}]=E[Q_{1,i}]E[K_{i,1}]=0$  
  $Var(Q_{1,i}K_{i,1})=(Var(Q_{1,i}+E[Q_{1,i}]^2))(Var(K_{i,1}+E[K_{i,1}]^2))-E[Q_{1,i}]^2E[K_{i,1}]^2=1$
- $E[A_{1,1}]=\sum\limits_{i=0}^{d_k}E[Q_{1,i}K_{i,1}]=0$
- $Var(A_{1,1})=\sum\limits_{i=0}^{d_k}Var(Q_{1,i}K_{i,1})=d_k$
- to scale each entry of A to a reasonable value which is suitable for softmax, we devide each entry with its std, which is equal to multiple with $\frac{1}{\sqrt{d_k}}$

